In [6]:
import contextlib
import os
import re
import sys

import linearmodels as lm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import s3fs
import seaborn as sns
import tabulate

sys.path.append("/Users/fgu/dev/projects/entropy")
import entropy.data.cleaners as cl
import entropy.data.creators as cr
import entropy.data.make_data as md
import entropy.data.selectors as sl
import entropy.data.validators as vl
import entropy.figures.figures as figs
import entropy.helpers.aws as ha
import entropy.helpers.data as hd
import entropy.helpers.helpers as hh
from entropy import config

sns.set_style("whitegrid")
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

fs = s3fs.S3FileSystem(profile="3di")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
dfs, dfl = hd.read_samples(["777", "000"])
hd.inspect(dfl)

Time for read_sample    : 1.44 seconds
Time for read_sample    : 0.31 seconds
(570, 28)


,id,date,user_id,amount,desc,merchant,tag_group,tag,user_female,user_postcode,user_registration_date,user_salary_range,user_yob,account_created,account_id,account_last_refreshed,account_provider,account_type,data_warehouse_date_created,data_warehouse_date_last_updated,debit,latest_balance,merchant_business_line,tag_auto,tag_manual,tag_up,updated_flag,ym
0,248375,2012-07-23,777,39.389999,9572 20jul 12 sacat marks spencer gemini gb cash back 30 00,NaN,spend,other_spend,0.0,wa1 4,2011-07-20,20k to 30k,1969.0,2011-07-20,262916,2020-06-25 22:55:00,natwest bank,current,2014-07-18,2017-10-23,True,2378.280029,personal,cash,groceries,groceries,u,201207
1,6755827,2013-06-21,777,7.410000,02jun a c 5781,NaN,spend,finance,0.0,wa1 4,2011-07-20,20k to 30k,1969.0,2011-07-20,262916,2020-06-25 22:55:00,natwest bank,current,2014-07-18,2017-08-14,True,2378.280029,account provider,interest charges,NaN,interest charges,u,201306


In [8]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170637 entries, 0 to 170636
Data columns (total 32 columns):
 #   Column                            Non-Null Count   Dtype         
---  ------                            --------------   -----         
 0   id                                170637 non-null  int32         
 1   date                              170637 non-null  datetime64[ns]
 2   user_id                           170637 non-null  int32         
 3   amount                            170637 non-null  float32       
 4   desc                              170637 non-null  category      
 5   merchant                          86573 non-null   category      
 6   tag_group                         136547 non-null  category      
 7   tag                               137429 non-null  category      
 8   user_female                       164497 non-null  float32       
 9   user_postcode                     170637 non-null  category      
 10  user_registration_date          

Entropy analysis data

In [3]:
filepath = "/Users/fgu/tmp/en/analysis_data_XX7.parquet"
dfa = ha.read_parquet(filepath)
hd.inspect(dfa)

(163,915, 23)


obs  balance_ca  balance_sa  sa_inflows  sa_outflows  \
user_id date                                                               
57      2012-04-30   66         0.0         NaN         NaN          NaN   
        2012-05-31   75         0.0         NaN         NaN          NaN   

                    sa_net_inflows  sa_scaled_inflows  sa_scaled_outflows  \
user_id date                                                                
57      2012-04-30             NaN                NaN                 NaN   
        2012-05-31             NaN                NaN                 NaN   

                    sa_scaled_net_inflows  total_monthly_spend  \
user_id date                                                     
57      2012-04-30                    NaN             6.784095   
        2012-05-31                    NaN             6.837290   

                    tag_spend_household  tag_spend_hobbies  tag_spend_retail  \
user_id date                                                                   
57      2012-04-30             0.400869           0.011305           0.00937   
        2012-05-31             0.469773           0.010719           0.00000   

                    tag_spend_services  tag_spend_other_spend  \
user_id date                                                    
57      2012-04-30            0.245824               0.067898   
        2012-05-31            0.317825              -0.014239   

                    tag_spend_finance  tag_spend_travel  \
user_id date                                              
57      2012-04-30           0.000837          0.228929   
        2012-05-31           0.001105          0.148343   

                    tag_spend_communication  tag_spend_motor  entropy_sptac  \
user_id date                                                                  
57      2012-04-30                 0.034967              0.0       2.425209   
        2012-05-31                 0.066473              0.0       2.363752   

                    log_income  user_female   age  
user_id date                                       
57      2012-04-30   10.023162          0.0  25.0  
        2012-05-31   10.023162          0.0  25.0

In [6]:
fs.ls("3di-data-mdb/raw")

['3di-data-mdb/raw/',
 '3di-data-mdb/raw/20200630_UserLoginsForNeedham.csv',
 '3di-data-mdb/raw/mdb_000.csv',
 '3di-data-mdb/raw/mdb_000.parquet',
 '3di-data-mdb/raw/mdb_777.csv',
 '3di-data-mdb/raw/mdb_777.parquet',
 '3di-data-mdb/raw/mdb_X77.csv',
 '3di-data-mdb/raw/mdb_X77.parquet',
 '3di-data-mdb/raw/mdb_XX7.csv',
 '3di-data-mdb/raw/mdb_XX7.parquet',
 '3di-data-mdb/raw/mdb_costa.csv']

### MDB user login data

In [93]:
@hh.timer
def read_logins(nrows=None):
    fp = "s3://3di-data-mdb/raw/20200630_UserLoginsForNeedham.csv"
    return ha.read_csv(fp, nrows=nrows, names=["user_id", "date"], parse_dates=["date"])


login_data = read_logins()
hd.inspect(login_data)

Time for read_logins    : 37.96 seconds
(7,521,416, 2)


,user_id,date
0,1,2015-07-23 14:28:00
1,1,2015-07-27 05:43:00


Proportion of users in dataset with login info

In [94]:
len(set(dfl.user_id) & set(login_data.user_id)) / dfl.user_id.nunique()

0.8884974458340673

Merge login data into transaction data

In [132]:
def make_daily_logins(login_data):
    login_data = login_data.copy()
    login_data["date"] = login_data.date.dt.round("d")
    login_data["logins"] = 1
    return login_data.groupby(["user_id", "date"]).logins.sum().reset_index()


def merge_logins(txn_data, login_data):
    txn_data = txn_data.copy()
    login_data = login_data.copy()
    merged = txn_data.merge(
        login_data, on=["user_id", "date"], how="left", validate="m:1"
    )
    merged["logins"] = merged.logins.fillna(0)
    return merged


def drop_pre_registration_data(df):
    """Drop all transactions from before a user registered with MDB.

    MDB receives up to three years of data after a user signs up. As
    a user can obviously not log in to their MDB account during that
    time, we drop these transactions.
    """
    return df[df.date >= df.user_registration_date]


txn_data = dfl
daily_logins = make_daily_logins(logins)
post_registration_txns = drop_pre_registration_data(txn_data)
df = merge_logins(post_registration_txns, daily_logins)
hd.inspect(df)

(9,539,059, 32)


,id,date,user_id,amount,desc,merchant,tag_group,tag,user_female,user_postcode,user_registration_date,user_salary_range,user_yob,account_created,account_id,account_last_refreshed,account_provider,account_type,data_warehouse_date_created,data_warehouse_date_last_updated,debit,latest_balance,merchant_business_line,tag_auto,tag_manual,tag_up,updated_flag,ym,balance,income,entropy_sptac,logins
0,1088993,2012-04-02,57,23.74,1982 01apr 12 the broadway fruiterer london gb,NaN,NaN,NaN,0.0,n8 9,2010-05-13,10k to 20k,1987.0,2016-10-30,273998,2015-08-15 21:37:00,natwest bank,current,2014-07-18,2017-10-23,True,NaN,NaN,NaN,NaN,NaN,u,201204,NaN,22542.600586,2.425209,0.0
1,1088994,2012-04-02,57,25.68,1982 01apr 12 waitrose 835 crouch end gb,waitrose,spend,household,0.0,n8 9,2010-05-13,10k to 20k,1987.0,2016-10-30,273998,2015-08-15 21:37:00,natwest bank,current,2014-07-18,2017-08-15,True,NaN,waitrose,"food, groceries, household",NaN,supermarket,u,201204,NaN,22542.600586,2.425209,0.0


In [145]:
def user_logins(df, freq="m"):
    period = pd.Grouper(key="date", freq=freq)
    return (
        df.groupby(["user_id", period])
        .logins.sum()
        .groupby(["user_id"])
        .agg(["mean", "median", "std", "size"])
    )


user_logins_data = user_logins(df)

In [146]:
hd.inspect(user_logins_data)

(5,204, 4)


,mean,median,std,size
user_id,,,,
57,8.160000,0.0,13.754500,100
107,11.605263,1.0,25.055181,76


In [147]:
user_logins_data.describe()

,mean,median,std,size
count,5204.000000,5204.000000,4992.000000,5204.000000
mean,6.809477,4.682840,7.701427,18.873751
std,13.708230,13.135486,11.108252,19.204074
min,0.000000,0.000000,0.000000,1.000000
25%,0.206643,0.000000,0.707107,5.000000
50%,1.838542,0.000000,3.714036,11.000000
75%,6.893939,3.000000,10.035591,25.000000
max,180.625000,181.500000,123.036580,100.000000
